In [22]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# sphinx_gallery_thumbnail_number = 6
from pathlib import Path
import re
import os
import subprocess

import eelbrain
from scipy.io import loadmat
import eelbrain.datasets._alice
from matplotlib import pyplot
import mne
from mne.channels import make_dig_montage
import numpy as np
from mne import find_events
from scipy.signal import resample

# Define the dataset root; this will use ~/Data/Alice, replace it with the
# proper path if you downloaded the dataset in a different location
DATA_ROOT = Path('FYP/music data')

# Define some paths that will be used throughout
STIMULUS_DIR = DATA_ROOT / 'diliBach_midi_4dryad'
EEG_DIR = DATA_ROOT / 'diliBach_4dryad_CND'

# Load one subject's raw EEG file
SUBJECT = 'Sub1'
LOW_FREQUENCY = 0.5
HIGH_FREQUENCY = 20


# In[2]:


import numpy as np
from pathlib import Path
import pretty_midi
import soundfile as sf  # Native WAV writing

SOUNDFONT_PATH = Path("path/to/your/soundfont.sf2")  # Optional but better results

def convert_midi_to_wav(fs=44100, gain=0.1, trim_thresh=1e-4):
    """Convert MIDI to WAV, normalize amplitude, and trim leading silence."""
    
    for i in range(1, 11):
        midi_file = STIMULUS_DIR / f'audio{i}.mid'
        wav_file = STIMULUS_DIR / f'{i}.wav'
        
        if not midi_file.exists():
            print(f"✗ {midi_file.name} not found")
            continue

        try:
            print(f"→ Converting {midi_file.name}...")
            midi = pretty_midi.PrettyMIDI(str(midi_file))

            # Render audio
            if SOUNDFONT_PATH.exists():
                audio = midi.fluidsynth(fs=fs, sf2_path=str(SOUNDFONT_PATH))
            else:
                audio = midi.synthesize(fs=fs)

            # Trim leading silence (based on amplitude threshold)
            abs_audio = np.abs(audio)
            nonzero_idx = np.where(abs_audio > trim_thresh)[0]
            if len(nonzero_idx) > 0:
                start_idx = nonzero_idx[0]
                audio = audio[start_idx:]
            else:
                print(f"⚠ No audible content in {midi_file.name}")
                continue

            # Normalize amplitude
            max_amp = np.max(np.abs(audio))
            if max_amp > 0:
                audio = audio * (gain / max_amp)

            # Save WAV
            sf.write(str(wav_file), audio, fs)
            duration = len(audio) / fs
            print(f"✓ Saved {wav_file.name} ({duration:.2f}s, max amp: {gain:.2f})")

        except Exception as e:
            print(f"✗ Failed to convert {midi_file.name}: {e}")




# Convert MIDI files to WAV
convert_midi_to_wav()

→ Converting audio1.mid...
✓ Saved 1.wav (158.04s, max amp: 0.10)
→ Converting audio2.mid...
✓ Saved 2.wav (153.10s, max amp: 0.10)
→ Converting audio3.mid...
✓ Saved 3.wav (119.29s, max amp: 0.10)
→ Converting audio4.mid...
✓ Saved 4.wav (134.50s, max amp: 0.10)
→ Converting audio5.mid...
✓ Saved 5.wav (164.73s, max amp: 0.10)
→ Converting audio6.mid...
✓ Saved 6.wav (198.31s, max amp: 0.10)
→ Converting audio7.mid...


/opt/homebrew/Caskroom/miniconda/base/envs/mtrf/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


✓ Saved 7.wav (172.17s, max amp: 0.10)
→ Converting audio8.mid...
✓ Saved 8.wav (181.21s, max amp: 0.10)
→ Converting audio9.mid...
✓ Saved 9.wav (133.17s, max amp: 0.10)
→ Converting audio10.mid...
✓ Saved 10.wav (177.10s, max amp: 0.10)
